In [1]:
import random as rn
import plotly.express as px
import pandas as pd
from tqdm.notebook import tqdm


In [2]:
def ensure_crypto_stays_crypto():
    for stonk in stonks_list:
        if stonk['name'] in ['Bit Coin', 'Ethereum','Doge Coin']: # crypto list
            stonk['phase'] = phase_crypto
            stonk['phase_number'] = 4
            
def stonk_update_every_round():
    # Updates the stonk price depending on its "phase"
    # if stonk price < lowerbound: 
    #   stonk price = lowerbound
    # Have the phase in the dictionary of the stonk
    # if stonk reaches var(stonk_max_value) in stonk['price']

    # Order stocks such that they are pretty...

#     for player in playersorder:
#         cur_stock_list = players[player]['Stonks']
#         ordered_stock_list = [stock for stock in stonks_list if stock in cur_stock_list ]
#         players[player]['Stonks'] = ordered_stock_list

    for stonk in stonks_list:
        if stonk_dist == "normal":
            phase1 = round(rn.normalvariate(phase1_mean , phase1_std) ,3)         # random float with a 3 decimal place
            phase2 = round(rn.normalvariate(phase2_mean , phase2_std) ,3)
            phase3 = round(rn.normalvariate(phase3_mean , phase3_std) ,3)
            phase_crypto = round(rn.normalvariate(phase_crypto_mean , phase_crypto_std) ,3)
        else:    # Note:  if stonk_dist == "uniform":
            phase1 = round(rn.uniform(phase1_bound[0],phase1_bound[1]) , 3)         # random float with a 3 decimal place
            phase2 = round(rn.uniform(phase2_bound[0],phase2_bound[1]) , 3)
            phase3 = round(rn.uniform(phase3_bound[0],phase3_bound[1]) , 3)
            phase_crypto = round(rn.uniform(phase_crypto_bound[0],phase_crypto_bound[1]) , 3)   

        if stonk['phase_number'] == 1:
            stonk['phase'] = phase1
        if stonk['phase_number'] == 2:
            stonk['phase'] = phase2
        if stonk['phase_number'] == 3:
            stonk['phase'] = phase3
        if stonk['phase_number'] == 4:
            stonk['phase'] = phase_crypto

        old_price = stonk['price']

        if (stonk['price'] < (bound_for_volatile_stonks//2)) and stonk['name'] not in ['Bit Coin', 'Ethereum','Doge Coin']:
            # Penny stonk go boom, avoid crypto cause its alrdy OP.
            stonk['price'] *= ((stonk['phase']**1.5) + 0.02)
        elif stonk['price'] < bound_for_volatile_stonks:
            # Small stonk, also go kinda boom
            stonk['price'] *= (stonk['phase']**1.2)
        elif stonk['price'] >= stonk_max_value//2:
            # Big stonk slow 
            stonk['price'] *= (stonk['phase']**0.8)
        else:
            # Normal stonk are normal
            stonk['price'] *= stonk['phase']

        stonk['price'] = int(stonk['price'])

        # print("old_price: ", old_price)
        # print("stonk['price']", stonk['price'])

        if stonk['price'] < stonk_minimum_value:
            stonk['price'] = int(stonk_minimum_value)
            
        elif stonk['price'] > stonk_max_value:
            stonk['price'] = int(stonk_drop_to_value)
        
        stonk['price_history'].append(int(stonk['price']))
        stonk['percent_increase_in_value'] = round(stonk['price']*100 /stonk['bought_price']  ,  1)
        stonk['percent_increase_pr_turn'] = round(stonk['price']*100 / old_price  ,  1)

        
def stonk_phase_change():
    global loan_interest

    for stonk in stonks_list:
        
        if stonk_dist == "normal":
            phase1 = round(rn.normalvariate(phase1_mean , phase1_std) ,3)         # random float with a 3 decimal place
            phase2 = round(rn.normalvariate(phase2_mean , phase2_std) ,3)
            phase3 = round(rn.normalvariate(phase3_mean , phase3_std) ,3)
            phase_crypto = round(rn.normalvariate(phase_crypto_mean , phase_crypto_std) ,3)
        else:    # Note:  if stonk_dist == "uniform":
            phase1 = round(rn.uniform(phase1_bound[0],phase1_bound[1]) , 3)         # random float with a 3 decimal place
            phase2 = round(rn.uniform(phase2_bound[0],phase2_bound[1]) , 3)
            phase3 = round(rn.uniform(phase3_bound[0],phase3_bound[1]) , 3)
            phase_crypto = round(rn.uniform(phase_crypto_bound[0],phase_crypto_bound[1]) , 3)

        if stonk['name'] in ['Bit Coin', 'Ethereum','Doge Coin']: # crypto list
            stonk['phase'] = phase_crypto
            stonk['phase_number'] = 4
            continue # skip the last bit as it's only for stonks and not crypto

        phase_choice = rn.randint(1,3)
        if phase_choice == 1:
            stonk['phase'] = phase1
            stonk['phase_number'] = 1
        elif phase_choice == 2:
            stonk['phase'] = phase2
            stonk['phase_number'] = 2
        elif phase_choice == 3:
            stonk['phase'] = phase3
            stonk['phase_number'] = 3
            
    for i in range(len(stonks_list) // 4):  
        stonk = rn.choice(stonks_list)
        stonk['phase'] = phase3
        stonk['phase_number'] = 3

In [3]:
stonk_start_value = 400                                             # DEFAULT:  start_value = 400  
stonk_minimum_value = stonk_start_value//2                          # DEFAULT:  stonk_minimum_value = start_value
stonk_max_value = 10000                                              # DEFAULT:  stonk_max_value = 5000   
bound_for_volatile_stonks = stonk_max_value//8                      # DEFAULT:  bound_for_volatile_stonks = stonk_max_value/4       
stonk_dist = "normal"                                               # DEFAULT:  stonk_dist = "normal" or "uniform"
#stonk_dist = "uniform"                                             # DEFAULT:  stonk_dist = "normal" or "uniform"
stonk_drop_to_value = stonk_max_value//10   

phase1_bound = (0.96,1.08)                                          # DEFAULT:  phase1_bound = (0.96 , 1.08)
phase2_bound = (0.85,1.10)                                          # DEFAULT:  phase2_bound = (0.85 , 1.07)
phase3_bound = (0.92,1.18)                                          # DEFAULT:  phase3_bound = (0.95 , 1.20)
phase_crypto_bound = (0.73,1.35)
phase_change_rounds = 7   


phase1_mean = (phase1_bound[0]+phase1_bound[1])/2     
phase2_mean = (phase2_bound[0]+phase2_bound[1])/2     
phase3_mean = (phase3_bound[0]+phase3_bound[1])/2     
phase_crypto_mean = (phase_crypto_bound[0]+phase_crypto_bound[1])/2

phase1_std = (phase1_mean - phase1_bound[0])/ 1.7       # 1.7 is an abritrary multiplier to determine a std.
phase2_std = (phase2_mean - phase2_bound[0])/ 1.7
phase3_std = (phase3_mean - phase3_bound[0])/ 1.7
phase_crypto_std = (phase_crypto_mean - phase_crypto_bound[0])/ 1.7

phase1 = round(rn.uniform(phase1_bound[0],phase1_bound[1]) , 2)
phase2 = round(rn.uniform(phase2_bound[0],phase2_bound[1]) , 2)
phase3 = round(rn.uniform(phase3_bound[0],phase3_bound[1]) , 2)
phase_crypto = round(rn.uniform(phase_crypto_bound[0],phase_crypto_bound[1]) , 2)

                                                                                                                                                                            
the_monopoly_stonk = {'name':'The Monopoly stonk' ,'price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None ,'price_history' : [], 'percent_increase_pr_round':0}
meat_industry_stonk = {'name':'Meat Industry stonk','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value':1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
bank_stonk = {'name':'Bank stonk','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value':1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0} 
tech_stonk = {'name':'Tech stonk','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value':1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0} 
clothes_stonk = {'name':'Clothes stonk','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
jewellery_store_stonk = {'name':'Jewellery store stonk','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
sports_stonk = {'name':'Sports stonk','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
food_chain_stonk = {'name':'Food Chain stonk','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
gold = {'name':'Gold Commodity','price': stonk_start_value , 'phase': phase1 , 'phase_number':1 ,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
silver = {'name':'Silver Commodity','price': stonk_start_value , 'phase': phase1 , 'phase_number':1 ,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
platinum = {'name':'Platinum Commodity','price': stonk_start_value , 'phase': phase1 , 'phase_number':1 ,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
oil = {'name':'Oil Commodity','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
bit_coin = {'name':'Bit Coin','price': stonk_start_value , 'phase': phase_crypto , 'phase_number':4,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
ethereum = {'name':'Ethereum','price': stonk_start_value , 'phase': phase_crypto , 'phase_number':4,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
doge_coin = {'name':'Doge Coin','price': stonk_start_value , 'phase': phase_crypto , 'phase_number':4,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}


stonks_list = [the_monopoly_stonk, meat_industry_stonk, bank_stonk,tech_stonk, clothes_stonk, jewellery_store_stonk
                ,sports_stonk, gold,silver,platinum, oil, food_chain_stonk, bit_coin, ethereum, doge_coin ]

In [4]:
count = 0

In [10]:
main_df = None
list_of_df = []

count += 1

stonk_start_value = 2000

for i in tqdm(range(1)):
    
    the_monopoly_stonk = {'name':'The Monopoly stonk' ,'price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None ,'price_history' : [], 'percent_increase_pr_round':0}
    meat_industry_stonk = {'name':'Meat Industry stonk','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value':1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
    bank_stonk = {'name':'Bank stonk','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value':1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0} 
    tech_stonk = {'name':'Tech stonk','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value':1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0} 
    clothes_stonk = {'name':'Clothes stonk','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
    jewellery_store_stonk = {'name':'Jewellery store stonk','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
    sports_stonk = {'name':'Sports stonk','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
    food_chain_stonk = {'name':'Food Chain stonk','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
    gold = {'name':'Gold Commodity','price': stonk_start_value , 'phase': phase1 , 'phase_number':1 ,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
    silver = {'name':'Silver Commodity','price': stonk_start_value , 'phase': phase1 , 'phase_number':1 ,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
    platinum = {'name':'Platinum Commodity','price': stonk_start_value , 'phase': phase1 , 'phase_number':1 ,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
    oil = {'name':'Oil Commodity','price': stonk_start_value , 'phase': phase1 , 'phase_number':1,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
    bit_coin = {'name':'Bit Coin','price': stonk_start_value , 'phase': phase_crypto , 'phase_number':4,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
    ethereum = {'name':'Ethereum','price': stonk_start_value , 'phase': phase_crypto , 'phase_number':4,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}
    doge_coin = {'name':'Doge Coin','price': stonk_start_value , 'phase': phase_crypto , 'phase_number':4,'bought_price':stonk_start_value , 'percent_increase_in_value': 1 , 'owner': None,'price_history' : [], 'percent_increase_pr_round':0}


    stonks_list = [the_monopoly_stonk, meat_industry_stonk, bank_stonk,tech_stonk, clothes_stonk, jewellery_store_stonk
                    ,sports_stonk, gold,silver,platinum, oil, food_chain_stonk, bit_coin, ethereum, doge_coin ]
    

    for i in range(100):
        if (i% 7 == 0 ) and i != 0:
            stonk_phase_change()
            ensure_crypto_stays_crypto()
        stonk_update_every_round()

    df = pd.DataFrame({ "{}".format(stonk['name']): stonk['price_history'] for stonk in stonks_list })
    df.reset_index(inplace = True)

    list_of_df.append(df.copy())

    
main_df = list_of_df[0].copy()
    
for idx in range(len(main_df)):
    for col in main_df:
        if col != "index":
            main_df.loc[idx,col] = sum([df.loc[idx,col] for df in list_of_df]) // len(list_of_df)


fig = px.line(main_df, x="index", y=[col for col in main_df if col != "index"], title='Stonks Simulation {}'.format(count))
fig.show()


    